## Train a language model to learn query semantics

In [1]:
# # Optional: you can set what GPU you want to use in a notebook like this.  
# # Useful if you want to run concurrent experiments at the same time on different GPUs.
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [31]:
use_cache = False

In [8]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

# EN = spacy.load('en_core_web_sm')
import en_core_web_sm
import pandas as pd
from sklearn.model_selection import train_test_split

# from general_utils import apply_parallel, flattenlist
EN = en_core_web_sm.load()

from ktext.preprocess import processor
import pandas as pd

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may i

In [2]:
use_cache = True

# Build Language Model From Docstrings

The goal is to build a language model using the docstrings, and use that language model to generate an embedding for each docstring.  

In [3]:
import pandas as pd
df = pd.read_pickle('./data/dataframe_processed.pkl')

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
def listlen(x):
    if not isinstance(x, list):
        return 0
    return len(x)

# separate functions w/o docstrings
# docstrings should be at least 3 words in the docstring to be considered a valid docstring

with_docstrings = df[df.docstring_tokens.str.split().apply(listlen) >= 3]
without_docstrings = df[df.docstring_tokens.str.split().apply(listlen) < 3]

In [5]:
df.head()

,nwo,path,function_name,lineno,original_function,function_tokens,docstring_tokens,api_sequence,tokenized_function_name
0,fnl/libfnl,src/fnl/nlp/dictionary.py,__init__,19,"def __init__(self, *leafs, **edges):\n self...",def __init__ self leafs edges self edges edges...,,self edges edges self leafs sorted leafs,init
1,fnl/libfnl,src/fnl/nlp/dictionary.py,__eq__,23,"def __eq__(self, other):\n if isinstance(ot...",def __eq__ self other if isinstance other Node...,,if isinstance other node return id self id oth...,eq
2,fnl/libfnl,src/fnl/nlp/dictionary.py,__repr__,29,def __repr__(self):\n return 'Node<leafs={}...,def __repr__ self return Node leafs edges form...,,"return node<leafs={}, edges={}> format self le...",repr
3,fnl/libfnl,src/fnl/nlp/dictionary.py,create_or_get,32,"def createOrGet(self, token):\n """"""\n\t\tCr...",def createOrGet self token if token in self ed...,create or get the node pointed to by ` token `...,if token self edges node self edges token else...,create or get
4,fnl/libfnl,src/fnl/nlp/dictionary.py,set_leaf,47,"def setLeaf(self, key, order):\n """"""\n\t\tS...",def setLeaf self key order self leafs append o...,store the ` key ` as a leaf of this node at po...,self leafs append order key self leafs sorted ...,set leaf


## Partition code by repository to minimize leakage between train, valid & test sets. 
Rough assumption that each repository has its own style.  We want to avoid having code from the same repository in the training set as well as the validation or holdout set.

In [6]:
grouped = with_docstrings.groupby('nwo')

In [20]:
# train, valid, test splits
train, test = train_test_split(list(grouped), train_size=0.87, shuffle=True, random_state=8081)
# train, valid = train_test_split(train, train_size=0.82, random_state=8081)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [21]:
train = pd.concat([d for _, d in train]).reset_index(drop=True)
# valid = pd.concat([d for _, d in valid]).reset_index(drop=True)
test = pd.concat([d for _, d in test]).reset_index(drop=True)

In [22]:
print(f'train set num rows {train.shape[0]:,}')
# print(f'valid set num rows {valid.shape[0]:,}')
print(f'test set num rows {test.shape[0]:,}')
print(f'without docstring rows {without_docstrings.shape[0]:,}')

train set num rows 1,222,657
test set num rows 179,248
without docstring rows 4,001,991


Preview what the training set looks like.  You can start to see how the data looks, the function tokens and docstring tokens are what will be fed downstream into the models.  The other information is important for diagnostics and bookeeping.

In [23]:
train.head()

,nwo,path,function_name,lineno,original_function,function_tokens,docstring_tokens,api_sequence,tokenized_function_name
0,euphorie/Euphorie,src/euphorie/content/module.py,searchable_text_indexer,191,@indexer(IModule)\ndef SearchableTextIndexer(o...,indexer IModule def SearchableTextIndexer obj ...,"index title , description and solution_direction",return join obj title strip_markup obj descrip...,searchable text indexer
1,euphorie/Euphorie,src/euphorie/content/module.py,tree_depth,276,"def tree_depth(obj):\n """"""Determine how dee...",def tree_depth obj submodules v for v in obj v...,determine how deeply nested a module structure...,submodules v v obj values i_question_container...,tree depth
2,euphorie/Euphorie,src/euphorie/content/module.py,item_depth,287,"def item_depth(item):\n """"""Return the surve...",def item_depth item from euphorie content surv...,return the survey depth of an item .,depth 0 for position aq_chain item else return...,item depth
3,euphorie/Euphorie,src/euphorie/content/module.py,_get_id,172,"def _get_id(self, orig_id):\n """"""Pick an id...",def _get_id self orig_id frame sys _getframe 1...,pick an i d for pasted content .,frame sys _getframe 1 ob frame f_locals get ob...,get id
4,euphorie/Euphorie,src/euphorie/content/module.py,update,216,"def update(self):\n """""" Set view attributes...",def update self self modules self _morph child...,set view attributes which list modules and ris...,self modules self _morph child child self cont...,update


In [24]:
import numpy as np
import scipy
print(np.__version__)
print(scipy.__version__)
import torch,cv2
from lang_model_utils import lm_vocab, load_lm_vocab, train_lang_model
from general_utils import save_file_pickle, load_file_pickle
import logging
from pathlib import Path
from fastai.text import *

source_path = Path('./data/processed_data/')

1.15.0
1.1.0


In [25]:
!which python

/opt/conda/bin/python


Preview what the raw data looks like: here are 10 docstrings

## Pre-process data for language model

We will use the class  `build_lm_vocab` to prepare our data for the language model

In [26]:
vocab = lm_vocab(max_vocab=50000,
                 min_freq=10)

# fit the transform on the training data, then transform
trn_flat_idx = vocab.fit_transform_flattened(train['docstring_tokens'])

Look at the transformed data

In [27]:
trn_flat_idx[:10]

array([  2, 153,  38,   4, 411,  15,   4, 591,   3,   2])

In [28]:
[vocab.itos[x] for x in trn_flat_idx[:10]]

['_xbos_',
 'generate',
 'all',
 'the',
 'functions',
 'in',
 'the',
 'program',
 '.',
 '_xbos_']

In [36]:
# apply transform to validation data
val_flat_idx = vocab.transform_flattened(test['docstring_tokens'])

Save files for later use

In [37]:
if not use_cache:
    vocab.save('./data/lang_model/vocab_v2.cls')
    save_file_pickle('./data/lang_model/trn_flat_idx_list.pkl_v2', trn_flat_idx)
    save_file_pickle('./data/lang_model/val_flat_idx_list.pkl_v2', val_flat_idx)

## Train Fast.AI Language Model

This model will read in files that were created and train a [fast.ai](https://github.com/fastai/fastai/tree/master/fastai) language model.  This model learns to predict the next word in the sentence using fast.ai's implementation of [AWD LSTM](https://github.com/salesforce/awd-lstm-lm).  

The goal of training this model is to build a general purpose feature extractor for text that can be used in downstream models.  In this case, we will utilize this model to produce embeddings for function docstrings.

In [38]:
vocab = load_lm_vocab('./data/lang_model/vocab_v2.cls')
trn_flat_idx = load_file_pickle('./data/lang_model/trn_flat_idx_list.pkl_v2')
val_flat_idx = load_file_pickle('./data/lang_model/val_flat_idx_list.pkl_v2')

In [39]:
if not use_cache:
    fastai_learner, lang_model = train_lang_model(model_path = './data/lang_model_weights_v2',
                                                  trn_indexed = trn_flat_idx,
                                                  val_indexed = val_flat_idx,
                                                  vocab_size = vocab.vocab_size,
                                                  lr=3e-3,
                                                  em_sz= 500,
                                                  nh= 500,
                                                  bptt=20,
                                                  cycle_len=1,
                                                  n_cycle=3,
                                                  cycle_mult=2,
                                                  bs = 200,
                                                  wd = 1e-6)
    
elif use_cache:    
    logging.warning('Not re-training language model because use_cache=True')

epoch      trn_loss   val_loss                                
    0      4.286136   4.302743  
    1      4.142069   4.161932                                
    2      4.0816     4.107555                                
    3      4.053363   4.077906                                
    4      4.029029   4.058487                                
    5      4.009092   4.040953                                


data/lang_model_weights_v2/models/langmodel_best.h5


    6      4.003826   4.029918  



In [40]:
if not use_cache:
    fastai_learner.fit(1e-3, 3, wds=1e-6, cycle_len=2)

epoch      trn_loss   val_loss                                
    0      3.994431   4.02065   
    1      3.93506    4.005463                                
    2      3.964569   4.005834                                
 36%|███▌      | 1387/3887 [01:48<03:15, 12.77it/s, loss=3.87]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 63%|██████▎   | 2433/3887 [03:11<01:54, 12.68it/s, loss=3.93]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    5      3.895408   3.985973                                



In [41]:
if not use_cache:
    fastai_learner.fit(1e-3, 2, wds=1e-6, cycle_len=3, cycle_mult=2)

 52%|█████▏    | 2004/3887 [02:37<02:28, 12.72it/s, loss=3.93]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 97%|█████████▋| 3781/3887 [04:59<00:08, 12.61it/s, loss=3.9] 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    2      3.876771   3.976019                                
 39%|███▉      | 1514/3887 [01:58<03:06, 12.72it/s, loss=3.89]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    4      3.906104   3.989113                                
    5      3.885839   3.978407                                
    6      3.851814   3.96768                                 
    7      3.833266   3.962222                                
    8      3.823778   3.96163                                 



In [42]:
if not use_cache:
    fastai_learner.fit(1e-3, 2, wds=1e-6, cycle_len=3, cycle_mult=10)

epoch      trn_loss   val_loss                                
    0      3.89211    3.984335  
    1      3.891199   3.960687                                
    2      3.835393   3.95829                                 
    3      3.916549   3.9925                                  
    4      3.919749   3.985729                                
    5      3.911503   3.982949                                
    6      3.87993    3.981166                                
    7      3.880064   3.981431                                
 79%|███████▉  | 3076/3887 [04:00<01:03, 12.82it/s, loss=3.84]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    9      3.893987   3.972651                                
 23%|██▎       | 902/3887 [01:10<03:53, 12.77it/s, loss=3.83]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 60%|██████    | 2347/3887 [03:03<02:00, 12.77it/s, loss=3.83]

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 60%|█████▉    | 2332/3887 [03:03<02:02, 12.69it/s, loss=3.71]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 89%|████████▊ | 3447/3887 [04:28<00:34, 12.83it/s, loss=3.73]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Save language model and learner

In [43]:
if not use_cache:
    fastai_learner.save('lang_model_learner_v2.fai')
    lang_model_new = fastai_learner.model.eval()
    torch.save(lang_model_new, './data/lang_model/lang_model_gpu_v2.torch')
    torch.save(lang_model_new.cpu(), './data/lang_model/lang_model_cpu_v2.torch')

# Load Model and Encode All Docstrings

Now that we have trained the language model, the next step is to use the language model to encode all of the docstrings into a vector. 

In [44]:
from lang_model_utils import load_lm_vocab
vocab = load_lm_vocab('./data/lang_model/vocab_v2.cls')
idx_docs = vocab.transform(train['docstring_tokens'], max_seq_len=30, padding=False)
lang_model = torch.load('./data/lang_model/lang_model_gpu_v2.torch', 
                        map_location=lambda storage, loc: storage)

In [45]:
lang_model.eval()

SequentialRNN(
  (0): RNN_Encoder(
    (encoder): Embedding(24902, 500, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(24902, 500, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(500, 500)
      )
      (1): WeightDrop(
        (module): LSTM(500, 500)
      )
      (2): WeightDrop(
        (module): LSTM(500, 500)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=500, out_features=24902, bias=False)
    (dropout): LockedDropout(
    )
  )
)

In [46]:
def list2arr(l):
    "Convert list into pytorch Variable."
    return V(np.expand_dims(np.array(l), -1)).cpu()

def make_prediction_from_list(model, l):
    """
    Encode a list of integers that represent a sequence of tokens.  The
    purpose is to encode a sentence or phrase.

    Parameters
    -----------
    model : fastai language model
    l : list
        list of integers, representing a sequence of tokens that you want to encode

    """
    arr = list2arr(l)# turn list into pytorch Variable with bs=1
    model.reset()  # language model is stateful, so you must reset upon each prediction
    hidden_states = model(arr)[-1][-1] # RNN Hidden Layer output is last output, and only need the last layer

    #return avg-pooling, max-pooling, and last hidden state
    return hidden_states.mean(0), hidden_states.max(0)[0], hidden_states[-1]


def get_embeddings(lm_model, list_list_int):
    """
    Vectorize a list of sequences List[List[int]] using a fast.ai language model.

    Paramters
    ---------
    lm_model : fastai language model
    list_list_int : List[List[int]]
        A list of sequences to encode

    Returns
    -------
    tuple: (avg, mean, last)
        A tuple that returns the average-pooling, max-pooling over time steps as well as the last time step.
    """
    n_rows = len(list_list_int)
    n_dim = lm_model[0].nhid
    avgarr = np.empty((n_rows, n_dim))
    maxarr = np.empty((n_rows, n_dim))
    lastarr = np.empty((n_rows, n_dim))

    for i in tqdm_notebook(range(len(list_list_int))):
        avg_, max_, last_ = make_prediction_from_list(lm_model, list_list_int[i])
        avgarr[i,:] = avg_.data.numpy()
        maxarr[i,:] = max_.data.numpy()
        lastarr[i,:] = last_.data.numpy()
        
    return avgarr, maxarr, lastarr

In [47]:
avg_hs, max_hs, last_hs = get_embeddings(lang_model, idx_docs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Do the same thing for the test set

In [51]:
idx_docs_test = vocab.transform(test['docstring_tokens'], max_seq_len=30, padding=False)
avg_hs_test, max_hs_test, last_hs_test = get_embeddings(lang_model, idx_docs_test)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

# Save Language Model Embeddings For Docstrings

In [50]:
savepath = Path('./data/lang_model_emb/')
np.save(savepath/'avg_emb_dim500_v2.npy', avg_hs)
np.save(savepath/'max_emb_dim500_v2.npy', max_hs)
np.save(savepath/'last_emb_dim500_v2.npy', last_hs)

In [55]:
# save the test set embeddings also
np.save(savepath/'avg_emb_dim500_test_v2.npy', avg_hs_test)
np.save(savepath/'max_emb_dim500_test_v2.npy', max_hs_test)
np.save(savepath/'last_emb_dim500_test_v2.npy', last_hs_test)

# Evaluate Sentence Embeddings

One popular way of evaluating sentence embeddings is to measure the efficacy of these embeddings in downstream tasks like sentiment analysis, textual similarity etc.  Usually you can use general-purpose benchmarks such as the examples outlined [here](https://github.com/facebookresearch/SentEval) to measure the quality of your embeddings.  However, since this is a very domain specific dataset - those general purpose benchmarks may not be appropriate.  Unfortunately, we have not designed downstream tasks that we can open source at this point.

In the absence of these downstream tasks, we can at least sanity check that these embeddings contain semantic information by doing the following:

1. Manually examine similarity between sentences, by supplying a statement and examining if the nearest phrase found is similar. 

2. Visualize the embeddings.

We will do the first approach, and leave the second approach as an exercise for the reader.  **It should be noted that this is only a sanity check -- a more rigorous approach is to measure the impact of these embeddings on a variety of downstream tasks** and use that to form a more objective opinion about the quality of your embeddings.

Furthermroe, there are many different ways of constructing a sentence embedding from the language model.  For example, we can take the average, the maximum or even the last value of the hidden states (or concatenate them all together).  **For simplicity, we will only evaluate the sentence embedding that is constructed by taking the average over the hidden states** (and leave other possibilities as an exercise for the reader). 

### Create search index using `nmslib` 

[nmslib](https://github.com/nmslib/nmslib) is a great library for doing nearest neighbor lookups, which we will use as a search engine for finding nearest neighbors of comments in vector-space.  

The convenience function `create_nmslib_search_index` builds this search index given a matrix of vectors as input.



In [58]:
from general_utils import create_nmslib_search_index
import nmslib
from lang_model_utils import Query2Emb
from pathlib import Path
import numpy as np
from lang_model_utils import load_lm_vocab
import torch

In [59]:
# Load matrix of vectors
loadpath = Path('./data/lang_model_emb/')
avg_emb_dim500 = np.load(loadpath/'avg_emb_dim500_test_v2.npy')

In [60]:
# Build search index (takes about an hour on a p3.8xlarge)
dim500_avg_searchindex = create_nmslib_search_index(avg_emb_dim500)

In [61]:
# save search index
dim500_avg_searchindex.saveIndex('./data/lang_model_emb/dim500_avg_searchindex.nmslib')

After you have built this search index with nmslib, you can do fast nearest-neighbor lookups.  We use the `Query2Emb` object to help convert strings to the embeddings: 

In [62]:
dim500_avg_searchindex = nmslib.init(method='hnsw', space='cosinesimil')
dim500_avg_searchindex.loadIndex('./data/lang_model_emb/dim500_avg_searchindex.nmslib')

In [63]:
lang_model = torch.load('./data/lang_model/lang_model_cpu_v2.torch')
vocab = load_lm_vocab('./data/lang_model/vocab_v2.cls')

q2emb = Query2Emb(lang_model = lang_model.cpu(),
                  vocab = vocab)

The method `Query2Emb.emb_mean` will allow us to use the langauge model we trained earlier to generate a sentence embedding given a string.   Here is an example, `emb_mean` will return a numpy array of size (1, 500).

In [64]:
query = q2emb.emb_mean('Read data into pandas dataframe')
query.shape

(1, 500)

**Make search engine to inspect semantic similarity of phrases**.  This will take 3 inputs:

1. `nmslib_index` - this is the search index we built above.  This object takes a vector and will return the index of the closest vector(s) according to cosine distance.  
2. `ref_data` - this is the data for which the index refer to, in this case will be the docstrings. 
3. `query2emb_func` - this is a function that will convert a string into an embedding.

In [70]:
class search_engine:
    def __init__(self, 
                 nmslib_index, 
                 ref_data, 
                 query2emb_func):
        
        self.search_index = nmslib_index
        self.data = ref_data
        self.query2emb_func = query2emb_func
    
    def search(self, str_search, k=3):
        query = self.query2emb_func(str_search)
        idxs, dists = self.search_index.knnQuery(query, k=k)
        for idx, dist in zip(idxs, dists):
            print(f'cosine dist:{dist:.4f}\n---------------\n', self.data[idx])

In [71]:
se = search_engine(nmslib_index=dim500_avg_searchindex,
                   ref_data = test['docstring_tokens'],
                   query2emb_func = q2emb.emb_mean)

## Manually Inspect Phrase Similarity

Compare a user-supplied query vs. vectorized docstrings on test set.  We can see that similar phrases are not exactly the same, but the nearest neighbors are reasonable.  

In [72]:
import logging
logging.getLogger().setLevel(logging.ERROR)

In [73]:
se.search('read csv into pandas dataframe')

cosine dist:0.0888
---------------
 read data from a pandas dataframe into a table .
cosine dist:0.0901
---------------
 read csv data from a file into a table .
cosine dist:0.0949
---------------
 read the whole text file into a pandas dataframe .


In [74]:
se.search('train a random forest')

cosine dist:0.0528
---------------
 train a network
cosine dist:0.0879
---------------
 trains a simple classification model .
cosine dist:0.0908
---------------
 train a model .


In [75]:
se.search('download files')

cosine dist:0.0925
---------------
 download humann2 database
cosine dist:0.0925
---------------
 download metaphlan2 database
cosine dist:0.0977
---------------
 download update data .


In [76]:
se.search('start webserver')

cosine dist:0.0463
---------------
 start sopcast service
cosine dist:0.0509
---------------
 start xephyr instance
cosine dist:0.0547
---------------
 start sopcast launcher


In [77]:
se.search('send out email notification')

cosine dist:0.0481
---------------
 send alert email
cosine dist:0.0664
---------------
 send email notifications .
cosine dist:0.0724
---------------
 send gntp registration


In [78]:
se.search('save pickle file')

cosine dist:0.0434
---------------
 save json file
cosine dist:0.0521
---------------
 save apj file
cosine dist:0.0521
---------------
 save furniture file
